## Importando dependências

In [ ]:
pip install --upgrade nbformat

In [ ]:
!pip install nbformat

In [ ]:
!pip install -U scikit_learn scipy missingpy

In [ ]:
!python -m pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

## Importando dados

In [ ]:
df_main = pd.read_csv('data/IM_230626_semNP.csv')

## Escolha de K

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
df_main = df_main.dropna(subset=["Patrimonio_Liquido"])
inadimplentes = df_main["Carteira_Direitos_Aquisicao_Inadimplentes"]
patrimonio_liquido = df_main["Patrimonio_Liquido"]
taxa_inadimplencia = inadimplentes / patrimonio_liquido

df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
df_inadimplencia = df_main[[
'Patrimonio_Liquido',
'Carteira_Direitos_Aquisicao_Inadimplentes']]

In [ ]:
SEED = 1224
np.random.seed(SEED)
pca_pipeline = Pipeline([('scaler', StandardScaler()),('PCA', PCA(n_components=2, random_state=SEED))])

Em relação à alta dimensionalidade, o algoritmo usa a distância euclidiana para determinar a associação de agrupamento, porém, quando temos uma alta dimensionalidade, pares de pontos começarão a ter distâncias muito semelhantes e não conseguiremos obter agrupamentos significativos. Para reduzir a quantidade de dados podemos utilizar a técnica chamada Análise de Componentes Principais (PCA).

Usando os dois módulos de auxílio, a padronização dos dados com o StandardScaler e a redução de dimensionalidade com o PCA, quando necessário, os resultados obtidos com o K-Means serão melhores do que apenas aplicar ele diretamente nos dados.

In [ ]:
genre_embbeding_pca = pca_pipeline.fit_transform(df_inadimplencia)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embbeding_pca)

In [ ]:
projection

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_pca = KMeans(n_clusters=5, verbose=True, random_state=SEED)

kmeans_pca.fit(projection)

df_inadimplencia['cluster_pca'] = kmeans_pca.predict(projection)

projection['cluster_pca'] = kmeans_pca.predict(projection)

In [ ]:
projection['Carteira_Direitos_Aquisicao_Inadimplentes'] = df_main['Carteira_Direitos_Aquisicao_Inadimplentes']

In [ ]:
projection

In [ ]:
fig = px.scatter(projection, x='x', y='y', color='cluster_pca', hover_data=['x','y', 'Carteira_Direitos_Aquisicao_Inadimplentes'])
fig.show()

uma forma de avaliarmos se a cluster foi bem separada ou não, é utilizarmos um explained variance do PCA.

Então vou colocar pca_pipeline[1].explained_variance_ratio_, estou passando a posição 1, porque a primeira, que seria 0, é do nosso StandardScaler, a 1 é do nosso PCA.

In [ ]:
pca_pipeline[1].explained_variance_ratio_.sum()

In [ ]:
pca_pipeline[1].explained_variance_.sum()

## Carteira financeiro

A métrica utilizada será a taxa de inadimplência, calculada pela carteira de direitos de aquisição inadimplentes dividido pelo patrimônio líquido

In [ ]:
df_main = df_main.dropna(subset=["Patrimonio_Liquido"])
inadimplentes = df_main["Carteira_Direitos_Aquisicao_Inadimplentes"]
patrimonio_liquido = df_main["Patrimonio_Liquido"]
taxa_inadimplencia = inadimplentes / patrimonio_liquido


df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)


In [ ]:
fundos = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos['Nome_Fundo'].unique())

selecionou-se os fundos cuja taxa de inadimplência fosse válida, ou seja, apenas aqueles que a taxa de inadimplência fosse diferente de 0 e NaN

In [ ]:
carteiras = [
    'Carteira',
    'Carteira_Industrial',
    'Carteira_Mercado_Imobiliario',
    'Carteira_Comercial_Total',
    'Carteira_Comercial',
    'Carteira_Comercial_Varejo',
    'Carteira_Arrendamento_Mercantil',
    'Carteira_Servicos_Total',
    'Carteira_Servicos',
    'Carteira_Servicos_Publicos',
    'Carteira_Servicos_Educacionais',
    'Carteira_Entretenimento',
    'Carteira_Agronegocio',
    'Carteira_Financeiro',
    'Carteira_Credito_Pessoal_Consignado',
    'Carteira_Credito_Corporativo',
    'Carteira_Middle_Market',
    'Carteira_Veiculos',
    'Carteira_Imobiliaria_Empresarial',
    'Carteira_Imobiliaria_Residencial',
    'Carteira_Outros_Financeiro',
    'Carteira_Cartao_Credito',
    'Carteira_Factoring',
    'Carteira_Factoring_Pessoal',
    'Carteira_Factoring_Corporativo',
    'Carteira_Setor_Publico',
    'Carteira_Precatorios',
    'Carteira_Creditos_Tributarios',
    'Carteira_Royalties',
    'Carteira_Outros_Setor_Publico',
    'Carteira_Acoes_Judiciais',
    'Carteira_Propriedade_Intelectual',
]

soma_carteiras = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=soma_carteiras, palette='magma')
plt.title('Taxas de Inadimplência por Carteira')
plt.ylabel('Taxa total de inadimplência')
plt.xlabel('Carteira')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

O gráfico foi plotado com todos os tipos de carteira, avaliando a taxa de inadimplência de cada segmento.

In [ ]:
soma_carteiras.head(5)

## K-means

O K-means é um algoritmo de agrupamento de dados que divide um conjunto de pontos em grupos (clusters) com base em suas características similares, buscando minimizar a variância dentro de cada grupo.

In [ ]:
from sklearn.cluster import KMeans

carteira_financeiro = 'Carteira_Financeiro'

fundos = df_main[df_main[carteira_financeiro].notna()]

X = fundos[[carteira_financeiro]]

kmeans = KMeans(n_clusters=3)
fundos['grupo'] = kmeans.fit_predict(X)

cores = ['#FFA500', '#00CED1', '#00008B']
fig, ax = plt.subplots(figsize=(12, 8))

for grupo in range(3):
    grupo_df = fundos[fundos['grupo'] == grupo]
    ax.scatter(grupo_df.index, grupo_df[carteira_financeiro],
               color=cores[grupo], label=f'Cluster {grupo}', s=100, alpha=0.7)

ax.set_title(f'{carteira_financeiro}', fontsize=16)
ax.set_xlabel('.', fontsize=14)
ax.set_ylabel('Inadimplência', fontsize=14)
ax.set_xticks([])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_facecolor('white')

plt.tight_layout()
plt.show()

In [ ]:
cluster1 = fundos[fundos['grupo'] == 1]

cluster1 = cluster1.sort_values(by='taxa_inadimplencia_series', ascending=False)

fundos_inadimplencia = cluster1.head(10)

for index, row in fundos_inadimplencia.iterrows():
    print(f'Nome do Fundo: {row["Nome_Fundo"]}')
    print(f'CNPJ do Fundo: {row["CNPJ"]}')
    print(f'Taxa de Inadimplência: {row["taxa_inadimplencia_series"]}')
    print(f'Data de competência: {row["Data_Competencia"]}')
    print('-' * 30)


Mostra as maiores taxas de inadimplência do cluster 1, ou seja do cluster que agrupa as maiores taxas de inadimplência dos dados.

#### Silhouette Score
O Silhouette Score avalia a coesão interna dos clusters e a separação entre os clusters. Ele varia de -1 a 1, com valores mais próximos de 1 indicando um bom agrupamento. Um valor negativo sugere que os pontos podem ter sido atribuídos ao cluster errado.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(X, fundos['grupo'])
f'Coeficiente de Silhueta: {silhouette_avg}'

#### Utilização do K-Means como Primeiro Modelo Candidato para o Problema

Na busca por uma solução eficiente para o problema em questão, a escolha inicial recaiu sobre o algoritmo K-Means. O K-Means é um método de clustering (agrupamento) amplamente utilizado na análise de dados e aprendizado de máquina. Sua aplicação visa agrupar dados em clusters, onde cada cluster contém pontos de dados que são mais semelhantes entre si do que com pontos de dados de outros clusters.

#### Justificativa para a Definição do K do Modelo

A determinação do número de clusters (K) é crucial para o sucesso do K-Means. Nesse contexto, foi empregada a métrica do coeficiente de silhueta (silhouette score) para encontrar o valor mais adequado de K. O coeficiente de silhueta mede a similaridade média entre um objeto e seu cluster em relação aos outros clusters, variando de -1 (representando uma alocação de cluster inadequada) a 1 (indicação de uma alocação ideal).

No teste realizado, o coeficiente de silhueta obteve um valor considerável de 0.9151350849612944. Esse resultado sugere que os clusters encontrados pelo K-Means são bem definidos e que a alocação de cada ponto de dados ao seu cluster é apropriada. Portanto, o valor de K utilizado nesse modelo é substancialmente apropriado para a tarefa, proporcionando uma segmentação eficaz dos dados.

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

scaler = StandardScaler()


dbscan = DBSCAN(eps=.1, min_samples=10)

carteira_financeiro = 'Carteira_Financeiro'

X = fundos[[carteira_financeiro]]
X = scaler.fit_transform(X)


fundos['grupo'] = dbscan.fit_predict(X)

fig, ax = plt.subplots(figsize=(12, 8))

for grupo in range(len(fundos['grupo'].unique())):
    grupo_df = fundos[fundos['grupo'] == grupo]
    ax.scatter(grupo_df.index, grupo_df[carteira_financeiro],
                label=f'Cluster {grupo}', s=100, alpha=0.7)

ax.set_title(f'{carteira_financeiro}', fontsize=16)
ax.set_xlabel('.', fontsize=14)
ax.set_ylabel('Inadimplência', fontsize=14)
ax.set_xticks([])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_facecolor('white')

plt.show()

In [ ]:
fundos['grupo'].value_counts()

In [ ]:
silhouette_avg = silhouette_score(X, fundos['grupo'])
f'Coeficiente de Silhueta: {silhouette_avg}'

# Escolha do modelo

Ao analisar a separação dos clusters da carteira financeira entre os algoritmos, o modelo escolhido foi o K-Means, pelo fato da sua separação ter sido uma das mais lógicas em relação ao nível de inadimplência, apesar do modelo não classificar os fundos relacionando-os diretamente com as features. Logo, é possível concluir que nenhum dos modelos candidatos apresentou uma separação muito clara ou significativa entre os clusters. Enquanto o K-Means realizou uma separação em 3 partes proporcionais em relação ao target, o DBSCAN acabou formando um cluster muito grande por conta de uma densidade de dados muito alta em determina, resultando em uma divisão muito desigual e também pouco significativa.